In [2]:
import argparse
import json
import os
from pathlib import Path
from threading import Thread

import numpy as np
import torch
import yaml
from tqdm import tqdm
from test import pred_label_onehot

from models.experimental import attempt_load
from utils.datasets import create_dataloader,create_dataloader_modified
from utils.general import coco80_to_coco91_class, check_dataset, check_file, check_img_size, check_requirements, \
    box_iou, non_max_suppression, scale_coords, xyxy2xywh, xywh2xyxy, set_logging, increment_path, colorstr,post_nms, nms_modified,nms_depthmap
from utils.metrics import ap_per_class, ConfusionMatrix
from utils.plots import plot_images, output_to_target, plot_study_txt,plot_images_modified
from utils.torch_utils import select_device, time_synchronized

import torchvision



if __name__ == '__main__':

    import os
    os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

    source = '/home/ypzhang/exp/comparison/images/test_02'
    weights = '/home/ypzhang/exp/yolov5holo/train/exp_bboxrescaled_weightedloss/weights/best.pt'
    # weights = '/Users/zhangyunping/PycharmProjects/yolov5holo/train/exp_depthmap/best.pt'
    view_image = True
    img_size = 512
    # project = '/content/drive/MyDrive/yoloV5/train/exp3'
    task = 'test'
    device = torch.device('cuda')
    set_logging()
    batch_size = 16

    # Load model
    model = attempt_load(weights, map_location=device)  # load FP32 model
    gs = max(int(model.stride.max()), 32)  # grid size (max stride)
    imgsz = check_img_size(img_size, s=gs)  # check img_size

    # half = device.type != 'cpu'  # half precision only supported on CUDA
    # if half:
    #     model.half()

    model.eval()
    nc = 256 # number of classes

    if device.type != 'cpu':
        model(torch.zeros(1, 3, imgsz, imgsz).to(device).type_as(next(model.parameters())))  # run once
    task = task if task in ('train', 'val', 'test') else 'val'  # path to train/val/test images
    dataloader = create_dataloader_modified(source, imgsz, batch_size, gs,
                                    pad=0.5, rect=True,
                                   prefix=colorstr(f'{task}: '),image_weights=True)[0]

    names = {k: v for k, v in enumerate(model.names if hasattr(model, 'names') else model.module.names)}


    mat = ConfusionMatrix(nc=256, conf=0, iou_thres=0.6)

    with torch.no_grad():
        for batch_i, (img, targets, paths, shapes) in enumerate(tqdm(dataloader)):
            # targets in the format [batch_idx, class_id, x,y,w,h]
            img = img.to(device, non_blocking=True)
            img = img.float()  # uint8 to fp16/32
            img /= 255.0  # 0 - 255 to 0.0 - 1.0
            nb, _, height, width = img.shape  # batch size, channels, height, width
            targets = targets.to(device)
            targets[:, 2:] *= torch.Tensor([width, height, width, height]).to(device)  # to pixels

            out_, train_out = model(img)  # inference and training outputs

            # # if would like to use one_hot for output
            # out = pred_label_onehot(out)
            # out = non_max_suppression(out)
            # out = post_nms(out,0.45)# list of anchors with [xyxy, conf, cls]


            # # if would like to use depthmap as the class directly
            # out = nms_modified(out_,obj_thre=0.8, iou_thres=0.5, nc=256) # list of anchors with [xyxy, conf, cls]
            out = nms_depthmap(out_,obj_thre=0.8, iou_thres=0.5, nc=256)
            # 因为用了torch自带的nms所以变成了xyxy


    # plot ----------------------------------------------------------------------------------------------------------------
            # list of detections, on (n,6) tensor per image [xyxy, conf, cls]
            plot_images_modified(img, targets, paths, fname='check.jpg', names=None)
            plot_images_modified(img, output_to_target(out),paths ,fname = 'check_pred.jpg',names=None)


    # update confusion matrix ----------------------------------------------------------------------------------------------
            for batch_idx in range(len(out)):
                labels = targets[targets[:,0].int()==batch_idx][:,1::] # class, x,y,w,h
                detections = out[batch_idx] # x,y,x,y,conf,cls
#                 detections[:,5] = detections[:,5].int()
                detections[:,5]  = torch.clamp(detections[:,5].int(),0,256)
                labels[:,1::] = xywh2xyxy(labels[:,1::])# class, x,y,x,y
                mat.process_batch(detections,labels)

    # Calculate accuracy      ----------------------------------------------------------------------------------------------

        mtx = mat.matrix
        accuracies = []
        for thred in [1, 5, 10, 20, 30]:
             # take prediction within this range as acceptable
            correct_match = 0
            total_num = np.sum(mtx[:, 0:256])
            for gt_cls in range(mtx.shape[1] - 1):
                correct_match += np.sum(mtx[max(0, gt_cls - thred):min(gt_cls + thred, mtx.shape[0] - 1), gt_cls])
            accuracy = correct_match / total_num
            accuracies.append(accuracy)
        s = ('%20s'*5) % ('Accuracy@0.08mm', 'Accuracy@0.39mm', 'Accuracy@0.78mm', 'Accuracy@1.5625mm','Accuracy@2.3438mm')
        print(s)
        pf = '{:20.2%}'*5
        print(pf.format(*accuracies))


Fusing layers... 
Model Summary: 224 layers, 7053910 parameters, 0 gradients, 16.3 GFLOPS
test: Scanning '/home/ypzhang/exp/comparison/labels/test_02' images and labels... 900 found, 0 missing, 0 empty, 0 corrupted: 100%|██████████| 900/900 [00:00<00:00, 1746.94it/s]
test: New cache created: /home/ypzhang/exp/comparison/labels/test_02.cache
100%|██████████| 57/57 [00:39<00:00,  1.45it/s]

     Accuracy@0.08mm     Accuracy@0.39mm     Accuracy@0.78mm   Accuracy@1.5625mm   Accuracy@2.3438mm
              32.09%              71.04%              76.49%              80.38%              82.82%
